In [1]:
import sys
from sys import platform
import matplotlib.pyplot as plt

In [2]:
if platform  == 'win32':
    sys.argv=["_", r"P:\rmahajan14\ndte_data\muni_data", "20170101", "20190930"]
if platform == 'linux' or platform == 'linux2':
    def full_path(p):
        return os.path.abspath(os.path.expanduser(p))
    sys.argv=["_", r"/mnt/d/DiDi_Ride_Data/data", r"/mnt/d/DiDi_Ride_Data/Ride_Data_Cache"]

In [3]:
from loader1 import *
from utils import *
from common import *
from experiment2 import *
# from sys import platform

import warnings
warnings.filterwarnings('ignore')

/mnt/d/DiDi_Ride_Data/Ride_Data_Cache/final_df_reg.msgpack exists


/mnt/c/Users/aksmi/Desktop/Drive/Fall '19/Capstone/capstone_didi/experiment2.py:173: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  df_final = get_final_df_reg()


In [4]:
merged_orders = merge_order_df()

/mnt/d/DiDi_Ride_Data/Ride_Data_Cache/merged_orders.msgpack exists


In [5]:
merged_orders

,order_id,ride_start_timestamp,ride_stop_timestamp,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,ride_duration,driver_id
0,00003ad9f837aca8ecdb9ae0729d0866,2016-11-01 14:35:44,2016-11-01 14:48:01,104.107101,30.699385,104.054980,30.688850,12.283333,64ee983f9979db09cb2586ec3893c467
1,0000c5bb7976b3dfe0fefa060265bdca,2016-11-01 15:06:22,2016-11-01 15:33:26,104.065170,30.702790,104.068280,30.666830,27.066667,19aa2c3cb2b05daf4ed06a3849c2c43c
2,0000f2c8aa21f2599a82923f317334bc,2016-11-01 19:38:49,2016-11-01 19:50:28,104.050859,30.687837,104.057100,30.662350,11.650000,15045dc2301012dda9cb8e98f4723efd
3,000102a3f8b31b3bc888572108888727,2016-11-01 11:22:02,2016-11-01 11:50:06,104.012450,30.622080,104.049190,30.755740,28.066667,53332bf0536f84453f0ab40a91774101
4,00017e395c1e449231b86f9149a793e5,2016-11-01 00:19:20,2016-11-01 00:34:41,104.092920,30.685830,103.990711,30.670803,15.350000,0dafbd548725f540988c5aa2554d782e
...,...,...,...,...,...,...,...,...,...
215023,fffe5967b6fdeab48c68f0ca2b330d61,2016-11-30 17:51:34,2016-11-30 18:47:20,103.897790,30.806530,104.042230,30.666520,55.766667,b7a5478a20a23993fde23f7cba244015
215024,fffe9e5c6d8030fd62448b8e53aacf47,2016-11-30 03:50:50,2016-11-30 04:22:22,104.073740,30.663010,103.928100,30.777570,31.533333,c6b7b37206ee6c6a38bb88a7c1655283
215025,fffeb486c95ea127189ae560038c6c3a,2016-11-30 12:27:08,2016-11-30 13:10:21,104.073912,30.695677,104.053270,30.765540,43.216667,d242eea3b731a2d1be5ecbd217fbe971
215026,ffff5cc2253bf6bca72dcb1e8a1958e6,2016-11-30 15:38:33,2016-11-30 15:49:03,104.113684,30.649077,104.099270,30.667460,10.500000,8e64c70c1c85b58dfcf79e7f939a8e17


### Remove rides with ride times greater than 3 hours

In [6]:
merged_orders = merged_orders[merged_orders.ride_duration <= 180]

In [7]:
# Check all diffs in start and end times
driver_start_times = merged_orders.loc[:, ['driver_id', 'ride_start_timestamp', 'ride_stop_timestamp', 'order_id']]\
            .drop_duplicates()
driver_start_times.sort_values(['driver_id', 'ride_start_timestamp'],
                               inplace=True)
driver_start_times['stop_time_shifted'] = driver_start_times.groupby(
    'driver_id')['ride_stop_timestamp'].shift(1)
driver_start_times['diff'] = driver_start_times[
    'ride_start_timestamp'] - driver_start_times['stop_time_shifted']

In [8]:
driver_start_times.sort_values(['driver_id', 'ride_start_timestamp']).head(100)

,driver_id,ride_start_timestamp,ride_stop_timestamp,order_id,stop_time_shifted,diff
175165,0000131d486b69eb77ab6e9e7cca9f4c,2016-11-01 11:57:05,2016-11-01 12:13:18,f727597b61fc2f0141aadecf2a3b5ee0,NaT,NaT
90347,0000131d486b69eb77ab6e9e7cca9f4c,2016-11-01 12:33:05,2016-11-01 12:49:04,804a1a47f2b10dc36b7e909d9b977f02,2016-11-01 12:13:18,00:19:47
156956,0000131d486b69eb77ab6e9e7cca9f4c,2016-11-01 12:58:45,2016-11-01 13:42:08,dd9b252638479fb784b705492bdf1a65,2016-11-01 12:49:04,00:09:41
2327,00002724a19c5f6a54ae8d60a378997e,2016-11-28 00:19:16,2016-11-28 00:54:04,03014caf7353cb92868d78b143bca071,NaT,NaT
64719,000042288ba964aa04d14869bd56efb6,2016-11-28 19:49:07,2016-11-28 20:00:58,504e0d59e3c0ae899a85c5e0551cb2b3,NaT,NaT
...,...,...,...,...,...,...
159084,0001681811178a09aa002f607c89735b,2016-11-28 21:57:31,2016-11-28 22:13:46,c465c5f4fd5cc991e08363014a94627b,2016-11-28 00:41:44,21:15:47
39259,00016e54648071044515d4ff3bd3ae18,2016-11-17 07:54:11,2016-11-17 08:39:17,30b9fd23e5665bf65486032031a11a36,NaT,NaT
134435,00016e54648071044515d4ff3bd3ae18,2016-11-17 13:19:47,2016-11-17 13:36:33,a671bbaf869d56a72c44153a536b654b,2016-11-17 08:39:17,04:40:30
58077,00016e54648071044515d4ff3bd3ae18,2016-11-17 13:40:27,2016-11-17 13:51:59,47dc5113cadb120b2e9d922ee2c96271,2016-11-17 13:36:33,00:03:54


In [9]:
driver_start_times_no_na = driver_start_times.dropna()

In [10]:
driver_start_times_no_na.shape

(4923622, 6)

In [11]:
driver_start_times_no_na['diff'] = driver_start_times_no_na['diff'].dt.total_seconds() / 60

In [12]:
driver_start_times_no_na.head()

,driver_id,ride_start_timestamp,ride_stop_timestamp,order_id,stop_time_shifted,diff
90347,0000131d486b69eb77ab6e9e7cca9f4c,2016-11-01 12:33:05,2016-11-01 12:49:04,804a1a47f2b10dc36b7e909d9b977f02,2016-11-01 12:13:18,19.783333
156956,0000131d486b69eb77ab6e9e7cca9f4c,2016-11-01 12:58:45,2016-11-01 13:42:08,dd9b252638479fb784b705492bdf1a65,2016-11-01 12:49:04,9.683333
5326,000042288ba964aa04d14869bd56efb6,2016-11-28 20:10:49,2016-11-28 20:20:25,06c7c82d8b15810ce80c47d0d3243ee1,2016-11-28 20:00:58,9.850000
118578,000042288ba964aa04d14869bd56efb6,2016-11-28 20:27:49,2016-11-28 21:02:33,929d619cd18a44b9ecdd259d71a13f7f,2016-11-28 20:20:25,7.400000
34798,000042288ba964aa04d14869bd56efb6,2016-11-28 22:32:06,2016-11-28 22:41:15,2b2880e2eb541443e747ecc649f67861,2016-11-28 21:02:33,89.550000


In [13]:
driver_start_times_no_na.describe()

,diff
count,4.923622e+06
mean,5.726774e+01
std,1.435211e+02
min,-1.638500e+02
25%,5.433333e+00
50%,1.033333e+01
75%,3.720000e+01
max,1.418300e+03


The mean difference between two consecutive pickup times is skewed right because of many drivers taking overnight breaks between two rides. A better estimate for the value of $\tau$ is the median difference of times between rides

In [52]:
mean_diff = driver_start_times_no_na['diff'].mean()

In [14]:
tau = driver_start_times_no_na['diff'].median()

This can be used to estimate the scale paramater for the decay by which the probability of a driver staying online decreases

$$\tau + \frac{1}{\lambda} = E(diff)$$

In [53]:
lmbd = 1./(mean_diff - tau)

In [55]:
shape = 1./lmbd

In [56]:
shape

46.93440354140374

In [71]:
# Defining function to determine online staying probability of a driver after time t:
from scipy.stats import expon
def idle_time_est(t, tau, shape, size):
    
    # sample from time diff distribution
    t_s = tau + np.random.exponential(scale=shape, size=size)
    
    indicator = t <= t_s
    indicator = 1 - indicator
    i_time = t - t_s
    idle_time = indicator * i_time
    
    return idle_time